In [52]:
from SpotifyPlayer import SpotifyPlayer
player = SpotifyPlayer()
player.authenticate()

Successfully authenticated


In [26]:
import joblib

kanyeId = "5K4W6rqBFWDnAN6FQUkS6x"
kanyeDiscography = player.getArtistDiscography(kanyeId)
joblib.dump(kanyeDiscography, 'joblib/kanyeDiscography.joblib')

allTracks = set()
for albumTracks in kanyeDiscography.values():
    allTracks |= albumTracks
print("Got discography")

Got discography


In [45]:
import time
runningPlaylistResults = {}
numIterations = 500
delay = 1 # seconds
probableKanyePlaylistQueries = ["rap", "hip hop", "pop"]

for i in range(numIterations):
    for genre in probableKanyePlaylistQueries:
        runningPlaylistResults = player.playlistQuery(genre, allTracks, runningPlaylistResults)
    time.sleep(delay)
    if i % 5 == 0:
        print("Done with", i)
        print(len(runningPlaylistResults))
        
    # don't wanna our token to expire
    if i % 10 == 0:
        oldLastIndex = player.lastIndex
        player = SpotifyPlayer()
        player.authenticate()
        player.lastIndex = oldLastIndex

joblib.dump(runningPlaylistResults, 'joblib/playlistResults.joblib')
runningPlaylistResults = runningPlaylistResults.values()

print("Done!")

# import glob
# import json
# playlistJSON = glob.glob('./playlists/*.json')

# def extractTrackURI(trackString):
#     i = trackString.rfind(":")
#     return trackString[(i+1):]


# runningPlaylistResults = []
# i = 0
# j = 0
# for playlistChunk in playlistJSON:
#     j += 1
#     with open(playlistChunk) as f:
#         playlists = json.load(f)["playlists"]
#         for playlist in playlists:
#             # {"tracks", "total"}
#             results = {}
#             results["total"] = playlist["num_tracks"]
#             results["tracks"] = []

#             for track in playlist["tracks"]:
#                 uri = extractTrackURI(track["track_uri"]) 
#                 if uri in allTracks:
#                     results["tracks"].append(uri)

#             if len(results["tracks"]) > 1:
#                 runningPlaylistResults.append(results)
#                 i += 1
    
# joblib.dump(runningPlaylistResults, 'joblib/millionPlaylist.joblib')
# print(i)
# print(j)

Done with 0
Successfully authenticated
retrying ...1secs
Done with 5
Done with 10
Successfully authenticated
retrying ...1secs
Done with 15
Done with 20
Successfully authenticated
retrying ...1secs
Done with 25
Done with 30
Successfully authenticated
Done with 35
Done with 40
Successfully authenticated
Done with 45
Done with 50
Successfully authenticated
Done with 55
Done with 60
Successfully authenticated
Done with 65
retrying ...1secs
Done with 70
Successfully authenticated
Done with 75
Done with 80
Successfully authenticated
Done with 85
Done with 90
Successfully authenticated
Done with 95
Done with 100
Successfully authenticated
Done with 105
Done with 110
Successfully authenticated
Done with 115
Done with 120
Successfully authenticated
Done with 125
Done with 130
Successfully authenticated
Done with 135
retrying ...1secs
Done with 140
Successfully authenticated
Done with 145


SpotifyException: http status: 401, code:-1 - https://api.spotify.com/v1/playlists/2jeAd6wYHX6va0DVJ5mfyw/tracks?fields=items.track.id:
 The access token expired

In [49]:
print(len(runningPlaylistResults))
joblib.dump(runningPlaylistResults, 'joblib/playlistResults.joblib')
runningPlaylistResults = runningPlaylistResults.values()

1263


In [50]:
import numpy as np

remainingTracks = set()

# initially track # occurrences, then use it to store the idf value
trackCounter = {}

for trackSet in runningPlaylistResults:
    remainingTracks |= set(trackSet['tracks'])
    for trackId in trackSet['tracks']:
        if trackId not in trackCounter:
            trackCounter[trackId] = 0
        trackCounter[trackId] += 1

numRemainingTracks = len(remainingTracks)
numPlaylists = len(runningPlaylistResults)
print("Found results for {}/{} songs".format(numRemainingTracks, len(allTracks)))

trackIndexer = {}
i = 0
# faster than an array for indexing
for trackId in remainingTracks:
    trackIndexer[trackId] = i
    i += 1
    trackCounter[trackId] = np.log(numPlaylists / trackCounter[trackId])

joblib.dump(trackIndexer, 'joblib/trackIndexer.joblib')

    
matrix = None
for playlistIdOutput in runningPlaylistResults:
    songArray = [0] * numRemainingTracks

    numSongsInPlaylist = playlistIdOutput['total']
    tf = 1.0 / numSongsInPlaylist

    for trackId in playlistIdOutput['tracks']:
        index = trackIndexer[trackId]
        idf = trackCounter[trackId]

        songArray[index] = tf * idf
    vector = np.array([songArray]).T
    if matrix is None:
        matrix = vector
    else:
        matrix = np.hstack((matrix, vector))

joblib.dump(matrix, 'joblib/matrix.joblib')


Found results for 193/250 songs


['joblib/matrix.joblib']

In [53]:
indexToTrack = np.array(sorted(trackIndexer, key=trackIndexer.get))

totalValueArray = np.array([-1 * sum([el ** 2 for el in row]) for row in matrix])
k = 10

topKSongs = np.argpartition(totalValueArray, kth=k)[:k]
sortedTopKSongs = topKSongs[np.argsort(totalValueArray[topKSongs])]
print("top songs")
i = 0
for trackId in indexToTrack[sortedTopKSongs]:
    trackInfo = player.sp._get("tracks/{}".format(trackId))
    print(str(i+1) + ".", trackInfo['name'])
    i += 1

top songs
1. I Love It (& Lil Pump)
2. Runaway
3. Monster
4. Stronger
5. Gold Digger
6. All Mine
7. Street Lights
8. POWER
9. Blame Game
10. Heartless


In [54]:
targetFeatures = [
    "acousticness",
    "danceability",
    "energy",
    "instrumentalness",
    "liveness",
    "loudness",
    "speechiness",
    "valence",
     "tempo"
]

featureMatrix = np.zeros((len(indexToTrack), len(targetFeatures)))

for i in range(len(indexToTrack) // 100 + 1):
    start = i * 100
    stop = (i + 1) * 100
    featureResponseChunk = player.sp._get('audio-features', ids=','.join(indexToTrack[start:stop]))


    featureArray = featureResponseChunk['audio_features']

    for songIndex in range(len(featureArray)):
        songFeatures = featureArray[songIndex]
        currentVector = []
        for targetFeature in targetFeatures:
            currentVector.append(songFeatures[targetFeature])
        featureMatrix[songIndex + start] = np.array(currentVector)

featureMatrix = np.hstack((matrix, featureMatrix))

joblib.dump(featureMatrix, 'joblib/featureMatrix.joblib')


['joblib/featureMatrix.joblib']

In [63]:
oldLastIndex

{'hip hop': 7400, 'pop': 7400, 'rap': 7400}

In [68]:
trackIdToName = {}
for trackId in indexToTrack:
    trackInfo = player.sp._get("tracks/{}".format(trackId))
    trackIdToName[trackId] = trackInfo['name']

In [72]:
numAlbumsAndSingles = len(kanyeDiscography)
numAlbums = 10
from sklearn.cluster import KMeans

matricesWithLabels = {
    "raw playlist matrix": matrix,
    "playlists + audio features": featureMatrix
}

for matrixLabel in matricesWithLabels:
    print("Computing for", matrixLabel,"\n")
    currentMatrix = matricesWithLabels[matrixLabel]
    knn = KMeans(numAlbums)
    labels = knn.fit_predict(currentMatrix)
    clusteredIndices = np.argsort(labels)
    i = -1
    for index in clusteredIndices:
        currentLabel = labels[index]
        if currentLabel != i:
            i = currentLabel
            print("Album",i)
        print("\t",trackIdToName[indexToTrack[index]])
    print("\n" * 3)

Computing for raw playlist matrix 

Album 0
	 Slow Jamz
	 I Wonder
	 Dark Fantasy
	 Good Life
	 Champion
	 Stronger
	 Lost In The World
	 New Slaves
	 Love Lockdown
	 Good Morning
	 Touch The Sky
	 The Glory
	 Through The Wire
	 Jesus Walks
	 All Falls Down
	 Flashing Lights
	 The New Workout Plan
	 Diamonds From Sierra Leone - Remix
	 POWER
	 Homecoming
	 Black Skinhead
	 Bound 2
	 Gold Digger
	 Everything I Am
	 Monster
	 Gorgeous
	 Heartless
	 Amazing
	 Blood On The Leaves
Album 1
	 Diamonds From Sierra Leone - Bonus Track
	 Guilt Trip
	 Dark Fantasy - Album Version (Edited)
	 RoboCop
	 Barry Bonds
	 Kids See Ghosts
	 All Falls Down - Album Version (Edited)
	 I Thought About Killing You
	 I Love It - (Freaky Girl) Edit
	 Homecoming
	 Intro
	 School Spirit
	 Reborn
	 Flashing Lights
	 Violent Crimes
	 Diamonds From Sierra Leone - Remix
	 Two Words
	 School Spirit
	 Heard 'Em Say
	 The Glory
	 Celebration
	 I Am A God
	 Never Let Me Down - Album Version (Edited)
	 Breathe In Breathe O